In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets,transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

transform= transforms.ToTensor()

train_data= datasets.FashionMNIST(root="./data", train=True, transform=transform, download=True)
test_data= datasets.FashionMNIST(root="./data", train=False, transform=transform, download=True)

In [ ]:
train_set, val_set = random_split(train_data, [50000,10000])

train_loader= DataLoader(train_set, batch_size=128, shuffle=True)
val_loader= DataLoader(val_set, batch_size=128, shuffle=False)
test_loader= DataLoader(test_data, batch_size=128, shuffle=False)

print(len(train_loader))
print(len(val_loader))
print(len(test_loader))



391
79
79


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
class FashionClassifer(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1= nn.Conv2d(1,16,kernel_size=3,stride=1,padding=1)
        self.bn1= nn.BatchNorm2d(16)
        self.relu1= nn.ReLU()

        self.conv2= nn.Conv2d(16,32, kernel_size=3,stride=2, padding=1)
        self.bn2= nn.BatchNorm2d(32)
        self.relu2= nn.ReLU()

        self.conv3= nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn3= nn.BatchNorm2d(64)
        self.relu3= nn.ReLU()

        self.conv4= nn.Conv2d(64,128, kernel_size=3, stride=2, padding=1)
        self.bn4= nn.BatchNorm2d(128)
        self.relu4= nn.ReLU()

        self.conv5= nn.Conv2d(128,256, kernel_size=3, stride=1, padding=1)
        self.bn5= nn.BatchNorm2d(256)
        self.relu5= nn.ReLU()

        self.gap = nn.AdaptiveAvgPool2d(1)   
        self.flatten = nn.Flatten()   
        self.drop = nn.Dropout(p=0.2)       
        self.fc = nn.Linear(256, 10)
        
    
    def forward(self, x):

        x = self.relu1(self.bn1(self.conv1(x)))
        x = self.relu2(self.bn2(self.conv2(x)))
        x = self.relu3(self.bn3(self.conv3(x)))
        x = self.relu4(self.bn4(self.conv4(x)))
        x=  self.relu5(self.bn5(self.conv5(x)))

        x = self.gap(x)        
        x = self.flatten(x)  
        x=  self.drop(x)
        x = self.fc(x)         
        return x


In [44]:
model= FashionClassifer().to(device)

criterion= nn.CrossEntropyLoss()

optimizer= torch.optim.Adam(model.parameters(), lr= 0.001)
scheduler= torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.5, patience=6)

num_epochs=30

best_loss= float("inf")

for epoch in range(num_epochs):

    model.train()
    running_loss=0

    for batch_idx, (images,labels) in enumerate(train_loader):

        images= images.to(device)
        labels= labels.to(device)

        output= model(images)
        loss= criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if (batch_idx+1)%100==0:

            print(f"epoch [{epoch+1}/{num_epochs}] | batch [{batch_idx+1}/{len(train_loader)}] | loss : {loss.item()} ")

    average_train_loss= running_loss/len(train_loader)
    print(f"epoch {epoch+1}/{num_epochs} avg train loss: {average_train_loss} ")


    model.eval()
    with torch.no_grad():
        running_val_loss=0

        for image,label in val_loader:
            image= image.to(device)
            label= label.to(device)

            output= model(image)
            loss= criterion(output,label)

            running_val_loss+= loss.item()

        avg_val_loss= running_val_loss/len(val_loader)
        print(f"epoch {epoch+1}/{num_epochs} avg val loss: {avg_val_loss} ")

    scheduler.step(avg_val_loss)

    if avg_val_loss < best_loss:
        best_loss= avg_val_loss
        torch.save(model.state_dict(), "best_model.pth")



epoch [1/30] | batch [100/391] | loss : 0.5807878971099854 
epoch [1/30] | batch [200/391] | loss : 0.3895679712295532 
epoch [1/30] | batch [300/391] | loss : 0.5024576783180237 
epoch 1/30 avg train loss: 0.5142346079773306 
epoch 1/30 avg val loss: 0.37431932363329057 
epoch [2/30] | batch [100/391] | loss : 0.3683091104030609 
epoch [2/30] | batch [200/391] | loss : 0.41545426845550537 
epoch [2/30] | batch [300/391] | loss : 0.2240244448184967 
epoch 2/30 avg train loss: 0.31849895196650035 
epoch 2/30 avg val loss: 0.3139192035681085 
epoch [3/30] | batch [100/391] | loss : 0.29745012521743774 
epoch [3/30] | batch [200/391] | loss : 0.20786608755588531 
epoch [3/30] | batch [300/391] | loss : 0.3195631504058838 
epoch 3/30 avg train loss: 0.273636883481994 
epoch 3/30 avg val loss: 0.31590429929238334 
epoch [4/30] | batch [100/391] | loss : 0.14042943716049194 
epoch [4/30] | batch [200/391] | loss : 0.26105040311813354 
epoch [4/30] | batch [300/391] | loss : 0.254272043704986

In [45]:
model.load_state_dict(torch.load("best_model.pth", map_location=device))
model.to(device)

total=0
correct=0

model.eval()
with torch.no_grad():
    for images,labels in test_loader:
        images= images.to(device)
        labels= labels.to(device)

        output=model(images)
        _,predicted= torch.max(output,1)

        total+= images.size(0)
        correct+= (predicted == labels).sum().item()

    accuracy= 100 * correct/total
    print(accuracy)

        



/tmp/ipykernel_21797/2011202425.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth", map_location=device))


90.69
